In [1]:
import os
import logging
import pylab
import numpy as np
from pypelid.survey.observation import Survey
from pypelid.survey.mask import Mask
logging.basicConfig(level=logging.DEBUG)

/Users/ben/.virtualenvs/pypelid/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


# Mask initialization

In [2]:
config = {
    'vm_density': 10  # number of randoms per square degree
     }

The Survey class loads the survey definition file which lists the coordinates of the pointings.  We can also specify the dithering pattern.  In this case there is no dithering.

In [3]:
survey = Survey(config=config,
                survey_file='survey.dat', 
                dither_offsets=[0,0],
                dither_angles=[0],
                verbose=True,
               )

> Computing dither positions and orientations of 42749 fields.


A polygon mask is constructed from the list of pointings and we save it to a file.  This takes 1 min for the full survey file with 42749 pointings.

In [4]:
mask_file = 'mask_full.pickle'
if not os.path.exists(mask_file):
    mask = survey.survey_mask()  # this takes ~1 min for the full survey file
    mask.dump(mask_file)
else:
    mask = Mask()
    mask.load(mask_file)

DEBUG:pypelid.survey.mask:import 683984 polygons
DEBUG:pypelid.survey.mask:count 68398: 9.999942 %
DEBUG:pypelid.survey.mask:count 136796: 19.999883 %
DEBUG:pypelid.survey.mask:count 205194: 29.999825 %
DEBUG:pypelid.survey.mask:count 273592: 39.999766 %
DEBUG:pypelid.survey.mask:count 341990: 49.999708 %
DEBUG:pypelid.survey.mask:count 410388: 59.999649 %
DEBUG:pypelid.survey.mask:count 478786: 69.999591 %
DEBUG:pypelid.survey.mask:count 547184: 79.999532 %
DEBUG:pypelid.survey.mask:count 615582: 89.999474 %
DEBUG:pypelid.survey.mask:count 683980: 99.999415 %
INFO:pypelid.survey.mask:Loaded 683984 polygons
INFO:pypelid.survey.mask:Wrote data to file mask_full.pickle.  time=58.768433


# Draw random sky coordinates in the survey

Points will be drawn from the survey area.  We will sample in patches of sky defined by healpix cells with the following nside parameter.

In [ ]:
nside = 2
pixel_list = np.arange(0,12*nside**2)

In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import healpy as hp

sc = ScalarMappable(Normalize(0, len(pixel_list)), cmap='jet')
hp.mollview(np.zeros(12) + float('nan'), cbar=False, title="Random catalogue")

for cell in pixel_list:
    lon, lat = mask.draw_random_position(dens=config['vm_density'], cell=cell, nside=nside)
    
    c = sc.to_rgba(cell)
    hp.visufunc.projscatter(lon, lat, c=c, s=5., lw=0, lonlat=True)

/sw/lib/python2.7/site-packages/healpy/pixelfunc.py:267: RuntimeWarning: invalid value encountered in less_equal
  return np.absolute(m - badval) <= atol + rtol * np.absolute(badval)
/sw/lib/python2.7/site-packages/numpy/ma/core.py:2249: RuntimeWarning: invalid value encountered in less_equal
  mabs(xnew - value), atol + rtol * mabs(value))
/sw/lib/python2.7/site-packages/healpy/projaxes.py:998: RuntimeWarning: invalid value encountered in less
  result.data[result.data<0]=0.0
/sw/lib/python2.7/site-packages/healpy/projaxes.py:999: RuntimeWarning: invalid value encountered in greater
  result.data[result.data>1]=1.0
DEBUG:pypelid.survey.mask:selected cell: 0 (nside 2)
DEBUG:pypelid.survey.mask:Building mask lookup tree
DEBUG:pypelid.survey.mask:Mask search radius: 0.002098
DEBUG:pypelid.survey.mask:pixel mask nside=256 order=ring
DEBUG:pypelid.survey.mask:Random sampling: npoints=570, ncells=1087
DEBUG:pypelid.survey.mask:done! elapsed time = 0.024326 sec
DEBUG:pypelid.survey.mask:Rand

In this plot the points in the random catalogue are colored by healpix cell.

In [ ]:
pylab.show()